In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import rouge

In [4]:
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [69]:
import json
import pandas as pd
import numpy as np
import itertools
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power


pd.set_option('display.max_colwidth', None)

In [6]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [21]:
data_path = '/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation'

In [7]:
train_args = pd.read_pickle('../../data/valid_args_with_scores.pkl')

In [8]:
train_topic_with_scored_kps = pd.read_pickle('../../data/valid_topic_keypoints_with_scores.pkl')

In [9]:
train_topics = train_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items]),
    'gt-kps' : lambda x: set([item for items in x for item in items]),
    #'gt-cand-kps': lambda x: set([item for items in x for item in items])
}).reset_index()

In [10]:
train_topics.columns

Index(['topic', 'stance', 'sents_with_scores', 'gt-kps'], dtype='object')

In [11]:
train_topics.head()

topic  stance  \
0        We should abandon the use of school uniform      -1   
1        We should abandon the use of school uniform       1   
2  We should abolish the right to keep and bear arms      -1   
3  We should abolish the right to keep and bear arms       1   
4                We should adopt an austerity regime      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [13]:
#Add to the candidate sentences from the topic, the topic key-points
train_topics['sents_with_scores'] = train_topics.apply(lambda row: list(row['sents_with_scores']) + train_topic_with_scored_kps[(train_topic_with_scored_kps.topic == row['topic']) & (train_topic_with_scored_kps.stance == row['stance'])]['scored_kps'].tolist()[0], axis=1)

In [56]:
models_list = [
     data_path + '/keypoint-analysis-sharedtask/final-experiment/roberta-large-final-model-fold-0-2023-07-03_14-50-42',
     data_path + '/keypoint-analysis-sharedtask/final-experiment/roberta-large-final-model-fold-1-2023-07-03_15-07-56',
     data_path + '/keypoint-analysis-sharedtask/final-experiment/roberta-large-final-model-fold-2-2023-07-03_15-22-51',
     data_path + '/keypoint-analysis-sharedtask/final-experiment/roberta-large-final-model-fold-3-2023-07-03_15-37-30',
     data_path + '/keypoint-analysis-sharedtask/final-experiment/roberta-large-final-model-fold-4-2023-07-03_15-52-27',
]
models = [SentenceTransformer(x) for x in models_list]

In [54]:
#dd = get_avg_match_matrix(models, ['sentence1', 'sentence2'], 'topic')

In [84]:
def gen_match_matrix(model, sents, topic, min_match_score=0):
    sents1 = [topic + ' <SEP> ' + x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0
    #print(sim_matrix)
    #print('--------;')
    return sim_matrix

def get_avg_match_matrix(models, sents, topic, min_match_score=0):
    sim_matrices = [gen_match_matrix(model, sents, topic, min_match_score) for model in models]
    #print(sim_matrices)
    final_matrix = np.mean(sim_matrices, axis=0)
    return final_matrix

def evalute_topic_by_rouge(row):
    ranked_sents = [[x[0]] for x in row['ranked_sents']][0:10]
    gt_kps = [[x] for x in row['gt-kps']]

    r_l = []
    for a,b in itertools.product(gt_kps, ranked_sents):
        #print(a, b)
        scores = evaluator.get_scores(a, b)
        r_l.append(scores['rouge-l']['f'])
    
    return np.mean(r_l)

def evalute_topic(row):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    #ranked_sents = [x for x in row['filtered_sents']]
    gt_kps_ranks = []
    for real_kp in row['gt-kps']:
        if real_kp in ranked_sents:
            gt_kps_ranks.append(ranked_sents.index(real_kp)+1)
        else:
            print('kp is not there...')
            gt_kps_ranks.append(len(ranked_sents)+1)
            
    #print(gt_kps_ranks)
    return 1/(sum(gt_kps_ranks)/len(gt_kps_ranks)) if len(gt_kps_ranks) > 0 else 0

In [91]:
def apply_page_rank(row, p=0.85, min_quality_score=0.5, min_match_score=0.5, min_len=5, max_len=20):
    cand_sents = [x for x in row['sents_with_scores'] if x[1] > min_quality_score and len(x[0].split()) < max_len and len(x[0].split()) > min_len]
    if len(cand_sents) == 0:
        return []
        print('empty')
    cands, cands_qualities = zip(*cand_sents)
    #print(cands)
    #print(cands_qualities)
    cands_qualities = [float(i)/sum(cands_qualities) for i in cands_qualities]
    
    cands_matching_mat = get_avg_match_matrix(models, cands, row['topic'], min_match_score=min_match_score)
    pr=pagerank(sparse.csr_matrix(np.array(cands_matching_mat)), personalize=np.array(cands_qualities), p=p) #it looks like modifying the initial probability doesn't help
    #pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cands, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])    

In [92]:
def filter_ranked_list(row, model, min_match=0.8):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    filtered_sents = []
    while len(filtered_sents) < 5:
        filtered_sents = []
        for i, s in enumerate(ranked_sents):
            if len(filtered_sents) == 0:
                filtered_sents.append(s)
                print('FFFF ' , s)
            else:
                matching_scores = get_avg_match_matrix(models, [s]+filtered_sents, row['topic'])
                max_sim = np.max(matching_scores[0][1:])
                #print(matching_scores)
                if max_sim < min_match:
                    print('SSSS ' , s)
                    filtered_sents.append(s)
        
        min_match = min_match + 0.1
        print(min_match)

    return filtered_sents

In [93]:
scores = []
for min_match_score in [0.2, 0.4, 0.6, 0.8, 1.0]:
    for p in [0.2, 0.4, 0.8, 1.0]:
        for min_quality_score in [0.2, 0.5, 0.8]:
            train_topics['ranked_sents'] = train_topics.apply(lambda row: apply_page_rank(row, p=p, min_quality_score=min_quality_score, min_match_score=min_match_score), axis=1)
            train_topics['scores'] = train_topics.apply(lambda row: evalute_topic_by_rouge(row), axis=1)
            score = np.mean(train_topics.scores.tolist())
            scores.append([p, min_quality_score, min_match_score, score])

/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.

In [95]:
scores_df = pd.DataFrame(scores, columns=['p', 'min-quality-score', 'min-match-score', 'score'])
scores_df.sort_values('score', ascending=False)

,p,min-quality-score,min-match-score,score
5,0.4,0.8,0.2,0.264123
17,0.4,0.8,0.4,0.263138
16,0.4,0.5,0.4,0.262617
15,0.4,0.2,0.4,0.262617
2,0.2,0.8,0.2,0.262257
0,0.2,0.2,0.2,0.262257
1,0.2,0.5,0.2,0.262257
3,0.4,0.2,0.2,0.261154
4,0.4,0.5,0.2,0.261154
56,0.8,0.8,1.0,0.260087


In [137]:
#train_topics['filtered_sents'] = train_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.3), axis=1)

-------------

##### Apply on test:

In [317]:
test_kps = pd.read_csv('../../KPA_2021_shared_task/test_data/key_points_test.csv')

In [318]:
test_args = pd.read_pickle('/workspace/ceph_data/keypoint-analysis-sharedtask/siamese-data/data-with-scores/test_args_with_scores.pkl')

In [319]:
test_args.head()

,arg_id,argument,topic,stance,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory since children don't spread the virus],"[(Routine child vaccinations isn't mandatory since children don't spread the virus, 0.7141945362091064)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be mandatory because children may not bear the side effects of it.],"[(Routine child vaccinations should not be mandatory because children may not bear the side effects of it., 0.871067464351654)]"
2,arg_0_2,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be necessary as children can't catch the disease],"[(Routine child vaccinations should not be necessary as children can't catch the disease, 0.8207754492759705)]"
3,arg_0_3,A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children,Routine child vaccinations should be mandatory,-1,[A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children],"[(A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children, 0.8951572775840759)]"
4,arg_0_4,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,"[As long as vaccines are not free of side effects, it cannot make them mandatory for our children.]","[(As long as vaccines are not free of side effects, it cannot make them mandatory for our children., 0.8620935082435608)]"


In [320]:
test_topics = test_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items])
}).reset_index()

In [321]:
test_topics['ranked_sents'] = test_topics.apply(lambda row: apply_page_rank(row, p=0.2, min_quality_score=0.8, min_match_score=0.8), axis=1)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [361]:
test_topics['ranked_kps'] = test_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.8), axis=1)

FFFF  vaccination should exclude children to avoid the side effects that can appear on them.
SSSS  parents should have the freedom to decide what they consider best for their children
SSSS  The child population has a low degree of vulnerability, so vaccination is not urgent yet
SSSS  When vaccines are mandatory, they can infringe on family religious choices
SSSS  Linking a measure as good as vaccination to coercive measures would cause serious harm
SSSS  Making child vaccination compulsory is not the the best way to obtain high vaccination rates
SSSS  vaccination of children should be limited as it affects child development
SSSS  child vaccinations is not mandatory because it may cost the country unnecessary funds.
SSSS  the poorest people may not be able to pay for the vaccine
SSSS  Forcing people to have their children vaccinated goes against free will.
SSSS  Vaccines contain heavy metals, they are toxic to children.
0.9
FFFF  child vaccinations should be mandatory to provide decent 

In [364]:
#adding gt-kps
test_topics['gt-kps']= test_topics.apply(lambda row: test_kps[(test_kps.topic==row['topic']) & (test_kps.stance==row['stance'])]['key_point'].tolist(), axis=1)

In [365]:
test_topics.head()

topic  stance  \
0                Routine child vaccinations should be mandatory      -1   
1                Routine child vaccinations should be mandatory       1   
2  Social media platforms should be regulated by the government      -1   
3  Social media platforms should be regulated by the government       1   
4                          The USA is a good country to live in      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [366]:
test_topics[['topic', 'stance', 'ranked_kps', 'gt-kps']].to_pickle('../../data/pagerank-test-keypoints.pkl')

In [370]:
keypoints_ds = []
c = 0
for i, row in test_topics.iterrows():
    for kp in row['ranked_kps'][0:5]:
        keypoints_ds.append([c, row['topic'], row['stance'], kp])
        c+=1
        
keypoints_df = pd.DataFrame(keypoints_ds, columns=['key_point_id', 'topic', 'stance', 'key_point'])

In [371]:
keypoints_df.sample(n=5).head()

,key_point_id,topic,stance,key_point
23,23,The USA is a good country to live in,-1,The United States has long been a hotbed of racism against minority cultures
2,2,Routine child vaccinations should be mandatory,-1,"The child population has a low degree of vulnerability, so vaccination is not urgent yet"
27,27,The USA is a good country to live in,1,The USA economy is one of the best in the world especially for job opportunities
22,22,The USA is a good country to live in,-1,The USA is a very capitalist country; it does not provide universal health care or education for its citizens.
29,29,The USA is a good country to live in,1,The USA has a well established and stable political system


In [372]:
ls ../../

KPA_2021_shared_task/  notes.txt      src-ipynb/
README.md              paraphrasing/  src-py/
data/                  siamese-nn/    track-2-keypoint-extraction/


In [373]:
keypoints_df.to_csv('../../track-2-keypoint-extraction/pagerank-generated-test-keypoints.csv', index=False)

In [374]:
keypoints_df.topic.value_counts()

Social media platforms should be regulated by the government    10
Routine child vaccinations should be mandatory                  10
The USA is a good country to live in                            10
Name: topic, dtype: int64